In [1]:
#load gemma7b
!pip uninstall torch -y
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install transformers accelerate bitsandbytes -qU
!pip install flash-attn --no-build-isolation
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import gc
import pandas as pd
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')

tokenizer=None
model=None
gc.collect()
torch.cuda.empty_cache()
modelName = "google/gemma-7b-it"

compute_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16

bnb_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_compute_dtype=compute_dtype,
)
tokenizer = AutoTokenizer.from_pretrained(modelName, token=hf_token)
model = AutoModelForCausalLM.from_pretrained(modelName, device_map="auto", token=hf_token, quantization_config=bnb_config)#, attn_implementation="flash_attention_2")


Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torch-2.3.0%2Bcu118-cp310-cp310-linux_x86_64.whl (839.6 MB)
  Using cached https://download.pytorch.org/whl/cu118/nvidia_nccl_cu11-2.20.5-py3-none-manylinux2014_x86_64.whl (142.9 MB)
  Using cached https://download.pytorch.org/whl/triton-2.3.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (168.1 MB)
  Using cached https://download.pytorch.org/whl/cu118/torch-2.2.1%2Bcu118-cp310-cp310-linux_x86_64.whl (819.2 MB)


Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [2]:

df_prompts = pd.read_csv("/content/custom_rewrite_prompts.csv")
df_ots = pd.read_csv("/content/mixed_ots.csv")


In [ ]:
#run inference
import time
import random
fname = "gemma7b_custom_prompts_rewrites.csv"
decoded_outputs = []
output_df = pd.DataFrame(columns=["original_text", "rewrite_prompt", "rewritten_text"])
output_df.to_csv(fname, index=False)
st=time.time()
model.eval()
ots=[]
ctr=0
template = "<start_of_turn>user\n{prompt}<end_of_turn>\n<start_of_turn>model\n"
prompts=df_prompts['rewrite_prompt'].to_list()
with torch.no_grad():
    for idx in range(len(prompts)):
        #choose a random original text
        ot_idx = random.randint(0, len(df_ots) - 1)
        ot = df_ots.iloc[ot_idx].values[0]
        prompt=f"{prompts[idx]}:\n{ot}"
        inputs = tokenizer(template.format(prompt=prompt), return_tensors="pt").to("cuda")
        outputs = model.generate(**inputs, max_new_tokens=inputs['input_ids'].shape[-1]*2)
        rewritten_text = tokenizer.decode(outputs[0][inputs['input_ids'].shape[-1]:], skip_special_tokens=True)
        decoded_outputs.append(rewritten_text)
        ots.append(ot)
        if (ctr + 1) % 20 == 0 or idx == len(prompts) - 1:  # Also save on the last iteration
            data_partial = {
                "original_text": ots[-20:] if ctr >= 19 else ots,
                "rewrite_prompt": prompts[idx - 19: idx + 1] if ctr >= 19 else prompts[:idx + 1],
                "rewritten_text": decoded_outputs[-20:] if ctr >= 19 else decoded_outputs
            }
            output_df_partial = pd.DataFrame(data_partial)
            output_df_partial.to_csv(fname, mode='a', header=False, index=False)
        print(f"ctr={ctr}")
        ctr+=1
        if ctr<=10:
            print(f"Prompt: {prompt}")
            print(f"Response: {rewritten_text}")

print(f"Elapsed time: {time.time()-st}")
print(f"Prompt: {prompt}")
print(f"Response: {rewritten_text}")


ctr=0
Prompt: Craft this passage as a cryptic crossword puzzle:
Vir is a bachelor from a small town working as a sous-chef in a five-star hotel in Mumbai, India. He is a handsome, charming, macho, idiot who flirts with, yearns for and lusts after any woman who is friendly, but seems unattainable. From his man-hating maid, whom he ogles as she wipes his floor, to the man-eating owner of a call centre who could wipe the floor with him, to the too-tall model, and the too-young pest down the corridor...from the Bengali-Beauty-with-Brains-and-Books to the married-flirt next door - Vir is suckered by them all.
With his ego, confusion and commitment ... it is so difficult to stop flattering and lying, to himself. The sometimes smart and sometimes silly observations of his all-male gang of cooks at the hotel kitchen is of little help until Vir is finally forced to grow up.
Response: ## Cryptic Crossword Puzzle

**Across:**

1. Handsome bachelor, working as a sous-chef in Mumbai, India. (8 lett